## Problem Statement

*Exercise 2.12 from Operations Research: Models and Methods by Jensen & Bard*

Ten jobs are to be completed by three workers during the next week. Each worker has a 40-hour work week. The times for the workers to complete the jobs are shown in the table. The values in the cells assume that each job is completed by a single worker; however, jobs can be shared, with completion times being determined proportionally If no entry exists in a particular cell, it means that the corresponding job cannot be performed by the corresponding worker. Set up and solve an LP model that will determine the optimal assignment of workers to jobs. The goal is to minimize the total time required to complete all the jobs.

| Workers \ Tasks |  1 |  2 |  3 |  4 |  5 |  6 |  7 |  8 |  9 | 10 |
|:---------------:|---:|---:|---:|---:|---:|---:|---:|---:|---:|---:|
| A               |  - |  7 |  3 |  - |  - | 18 | 13 |  6 |  - |  9 |
| B               | 12 |  5 |  - | 12 |  4 | 22 |  - | 17 | 13 |  - |
| C               | 18 |  - |  6 |  8 | 10 |  - | 19 |  - |  8 | 15 |

## Import

In [ ]:
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po

## Define Data

In [ ]:
workers = {'A', 'B', 'C'}

tasks = set(range(1, 11))

c = {
    ('A',  2):  7,
    ('A',  3):  3,
    ('A',  6): 18,
    ('A',  7): 13,
    ('A',  8):  6,
    ('A', 10):  9,
    ('B',  1): 12,
    ('B',  2):  5,
    ('B',  4): 12,
    ('B',  5):  4,
    ('B',  6): 22,
    ('B',  8): 17,
    ('B',  9): 13,
    ('C',  1): 18,
    ('C',  3):  6,
    ('C',  4):  8,
    ('C',  5): 10,
    ('C',  7): 19,
    ('C',  9):  8,
    ('C', 10): 15,
}

max_hours = 40

## Model
Define $W$ as the set of workers and $T$ as the sets of tasks. Also, define $c_{wt}$ as the number of hours worker $w$ requires to complete task $t$. (Note that we do not explicitly prohibit a worker from completiting as task; rather, we make the cost arbitrarily large if worker $w$ is unable to perform task $t$.) Let $x_{wt}$ be the proportion of task $t$ that is completed by worker $j$. Let $H$ be the max number of hours that any single worker may log in a week. We formulate as follows.


$$
\begin{alignat*}{3}
\text{minimize  }  & \sum_{w \in W} \sum_{t \in T} c_{wt} x_{wt} && \\
\text{subject to  }
& \sum_{t \in T} c_{wt} x_{wt} \le H,
&& \qquad \forall w \in W \\
& \sum_{w \in W} x_{wt} = 1
&& \qquad \forall t \in T \\
& 0 \le x_{wt} \le 1,
&& \qquad \forall w \in W, \forall t \in T
\end{alignat*}
$$

## Implement

In [ ]:
model = pe.ConcreteModel()

In [ ]:
model.workers = pe.Set(initialize=workers)
model.tasks = pe.Set(initialize=tasks)

In [ ]:
model.c = pe.Param(model.workers, model.tasks, initialize=c, default=1000)
model.max_hours = pe.Param(initialize=max_hours)

In [ ]:
model.x = pe.Var(model.workers, model.tasks, domain=pe.Reals, bounds=(0, 1))

In [ ]:
expr = sum(model.c[w, t] * model.x[w, t]
           for w in model.workers for t in model.tasks)
model.objective = pe.Objective(sense=pe.minimize, expr=expr)

In [ ]:
model.tasks_done = pe.ConstraintList()
for t in model.tasks:
    lhs = sum(model.x[w, t] for w in model.workers)
    rhs = 1
    model.tasks_done.add(lhs == rhs)

In [ ]:
model.hour_limit = pe.ConstraintList()
for w in model.workers:
    lhs = sum(model.c[w, t] * model.x[w, t] for t in model.tasks)
    rhs = model.max_hours
    model.hour_limit.add(lhs <= rhs)

## Solve and Postprocess

In [ ]:
solver = po.SolverFactory('gurobi')
results = solver.solve(model, tee=True)

In [ ]:
df = pd.DataFrame(index=pd.MultiIndex.from_tuples(model.x, names=['w', 't']))
df['x'] = [pe.value(model.x[key]) for key in df.index]
df['c'] = [model.c[key] for key in df.index]

In [ ]:
(df['c'] * df['x']).unstack('t')

In [ ]:
(df['c'] * df['x']).groupby('w').sum().to_frame()

In [ ]:
df['x'].groupby('t').sum().to_frame().T